In [1]:
import ipywidgets as ipw
import pandas_datareader as pdr
import datetime
import numpy as np
import scipy as sp
import pandas as pd
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from ipywidgets.embed import embed_minimal_html
from ipywidgets.embed import embed_data
from ipywidgets.embed import embed_data
import matplotlib.pyplot as plt 
import statsmodels.api as sm
from dateutil.parser import parse

# Defining Function

In [2]:
def TRADE(stk,mtd,st_dte,end_dte,short_avg,long_avg,vol_tf,out):
    strt=parse(st_dte)
    strt=strt.strftime('%Y/%m/%d')
    
    if end_dte=='now':
        end=datetime.datetime.now()
    else:
        end=parse(end_dte)
        end=end.strftime('%Y/%m/%d')
    
    df=pdr.get_data_yahoo(stk, strt, end)
    
    sig_df=pd.DataFrame(index=df.index)
    sig_df['Close']=df.Close
    sig_df['signal']=0
    
    sig_df['short_moving_average']=df.Close.rolling(window=short_avg,min_periods=1, center=False).mean()
    sig_df['long_moving_average']=df.Close.rolling(window=long_avg,min_periods=1, center=False).mean()
    
    if mtd=='Mean Reversion Strategy':
        sig_df['signal'][short_avg:]=np.where(sig_df['short_moving_average'][short_avg:]<
                                          sig_df['long_moving_average'][short_avg:],1,0)
        sig_df['position']=sig_df['signal'].diff()
    elif mtd=='Crossover Strategy':
        sig_df['signal'][short_avg:]=np.where(sig_df['short_moving_average'][short_avg:]>
                                          sig_df['long_moving_average'][short_avg:],1,0)
        sig_df['position']=sig_df['signal'].diff()
    elif mtd=='Statistics':
        price=df[['Adj Close']]
        diff=price.pct_change()
        diff.fillna(0, inplace=True)
        vol = diff.rolling(vol_tf).std() * np.sqrt(vol_tf) 
        df['Price']=price
        df['Volatility']=vol
        df['Short Moving Average']=sig_df['short_moving_average']
        df['Long Moving Average']=sig_df['long_moving_average']
        sig_df=df
        
    %matplotlib agg
    %matplotlib agg
    
    fig=plt.figure()
    axis_font = {'fontname':'Arial', 'size':'24'}
    if mtd=='Mean Reversion Strategy':
        ax1=fig.add_subplot(111)
        ax1.plot(sig_df.index,df['Close'], color='r', lw=1)
        ax1.plot(sig_df.index,sig_df['short_moving_average'], color='b', lw=2)
        ax1.plot(sig_df.index,sig_df['long_moving_average'], color='y', lw=2)
        ax1.plot(sig_df.loc[sig_df.position==1].index,sig_df.short_moving_average[sig_df.position==1],
         '^', markersize=10, color='m')
        ax1.plot(sig_df.loc[sig_df.position==-1].index,sig_df.short_moving_average[sig_df.position==-1],
         'v', markersize=10, color='k')
        plt.title('Mean Reversion Strategy',fontsize=40)
        plt.gca().legend(('Closing Price','Short Moving Avg','Long Moving Avg','Buy','Sell'),fontsize=16)
        plt.xlabel("Date", **axis_font)
        plt.ylabel("Price", **axis_font)
    elif mtd=='Crossover Strategy':
        ax1=fig.add_subplot(111)
        ax1.plot(sig_df.index,df['Close'], color='r', lw=1)
        ax1.plot(sig_df.index,sig_df['short_moving_average'], color='b', lw=2)
        ax1.plot(sig_df.index,sig_df['long_moving_average'], color='y', lw=2)
        ax1.plot(sig_df.loc[sig_df.position==1].index,sig_df.short_moving_average[sig_df.position==1],
         '^', markersize=10, color='m')
        ax1.plot(sig_df.loc[sig_df.position==-1].index,sig_df.short_moving_average[sig_df.position==-1],
         'v', markersize=10, color='k')
        plt.title('Crossover Trading Strategy',fontsize=40)
        plt.gca().legend(('Closing Price','Short Moving Avg','Long Moving Avg','Buy','Sell'),fontsize=16)
        plt.xlabel("Date", **axis_font)
        plt.ylabel("Price", **axis_font)
    elif mtd=='Statistics':
        ax1=fig.add_subplot(131)
        plt.plot(df.index,df['Adj Close'],color='r',lw=1)
        plt.plot(df.index,df['Short Moving Average'],color='b',lw=1)
        plt.plot(df.index,df['Long Moving Average'],color='y',lw=1)
        ax1.set_xlabel("Date", **axis_font)
        ax1.set_ylabel("Price", **axis_font)
        ax1.set_title("Stock Price",fontsize=20)
        plt.gca().legend(('Adjusted Price','Small Moving Average','Long Moving Average'),fontsize=16)
        plt.grid(True)

        ax2=fig.add_subplot(132)
        plt.hist(diff['Adj Close'])
        ax2.set_xlabel("Percent Change", **axis_font)
        ax2.set_title("Daily Percentage Change",fontsize=20)
        plt.grid(True)

        ax3=fig.add_subplot(133)
        plt.plot(vol.index, vol)
        ax3.set_xlabel("Date", **axis_font)
        ax3.set_title("Volatility",fontsize=20)
    
    plt.grid(True)
    fig.set_size_inches(18.5, 10.5)
    plt.rcParams.update({'font.size': 10})
    
    if out=='Plot':
        return fig
    elif out=='Data':
        return sig_df

# Function Example

The first two lines provides sample code for gathering data of Microsoft stock and Apple stock from $1^{st}$ January 2006 to 
$1^{st}$ January 2012. The last two line create plots from implementing the Mean Reversion Strategy and Crossover Strategy
from $1^{st}$ January 2006 to $1^{st}$ January 2012 for Microsoft and Apple stock, respectively. 

In [3]:
msft_data=TRADE('MSFT','Statistics','2006-01-01','2012-01-01',40,100,40,'Data')
aapl_data=TRADE('AAPL','Statistics','2006-01-01','2012-01-01',40,100,40,'Data')

msft_plot=TRADE('MSFT','Mean Reversion Strategy','2006-04-01','2012-01-01',40,100,40,'Plot')
aapl_plot=TRADE('AAPL','Crossover Strategy','2006-04-01','2012-01-01',40,100,40,'Plot')

<ipython-input-2-c0767eae186c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sig_df['signal'][short_avg:]=np.where(sig_df['short_moving_average'][short_avg:]<
<ipython-input-2-c0767eae186c>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sig_df['signal'][short_avg:]=np.where(sig_df['short_moving_average'][short_avg:]>


# User Interface

In this section I construct a user interface to apply the above function seemlessly. Running the next line will result in a user interface which provides 3 boxes, 3 sliders and two toggle buttons. The boxes correspond to the stock ticker, start and end date. The sliders control the time period for volatility, short run and long run moving averages. You can select the method and output using the two toggle buttons. 

In [6]:
box_layout = Layout(border='solid')

nbox=ipw.Text(
    value='AAPL',
    placeholder='Type something',
    description='Stock Ticker',
    disabled=False
)

method=ipw.RadioButtons(
    options=['Crossover Strategy', 'Mean Reversion Strategy', 'Statistics'],
    description='Method',
    disabled=False
)

beg=ipw.Text(
    value='YYYY-MM-DD',
    placeholder='Type something',
    description='Start Date:',
    disabled=False
)

fin=ipw.Text(
    value='YYYY-MM-DD',
    placeholder='Type something',
    description='End Date:',
    disabled=False
)

sma=ipw.IntSlider(
    value=40,
    min=1,
    max=365.0,
    step=1,
    description='Short Moving Average',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style={'description_width': 'initial'},
    layout=Layout(width='50%')
)

lma=ipw.IntSlider(
    value=100,
    min=1,
    max=365.0,
    step=1,
    description='Long Moving Average',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style={'description_width': 'initial'},
    layout=Layout(width='50%')
)

voli=ipw.IntSlider(
    value=90,
    min=1,
    max=365.0,
    step=1,
    description='Volatility timeframe',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style={'description_width': 'initial'},
    layout=Layout(width='50%')
)

dis=ipw.ToggleButtons(
    options=['Data', 'Plot'],
    description='Output',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)
GUI=interact_manual(TRADE,stk=nbox,mtd=method,st_dte=beg,end_dte=fin,short_avg=sma,long_avg=lma,vol_tf=voli,out=dis)

interactive(children=(Text(value='AAPL', description='Stock Ticker', placeholder='Type something'), RadioButto…